# Experiments on the effect of learning parameters and mechanisms:
- learning rate
- batch size
- use of callbacks

In [2]:
from src import util
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print("GPUs Available: ", tf.config.list_physical_devices('GPU'))


GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
sensor_data_trainval, sequence_ids_trainval, labels_trainval = util.ingest_sensor_data('train.csv', 'train_labels.csv')
sensor_data_test, sequence_ids_test, _ = util.ingest_sensor_data('test.csv')

sensor_data_train, sequence_ids_train, labels_train, sensor_data_val, sequence_ids_val, labels_val = util.validation_split(training_fraction=0.7, sensor_data=sensor_data_trainval, sequence_ids=sequence_ids_trainval, labels=labels_trainval)

(25968, 60, 13) 25968
(12218, 60, 13) 12218


In [6]:
lstm_dim = 64
dense_dim=64
lstm_dropout=0.15

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=lstm_dim, input_shape=(60,13), dropout=lstm_dropout),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 10))
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


history=model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=50, callbacks=[lr_schedule])

plt.semilogx(history.history["lr"], history.history["loss"])
#plt.axis([1e-8, 1e-4, 0, 30])
plt.show()

Epoch 1/50
650/650 [==============================] - 14s 16ms/step - loss: 0.6879 - accuracy: 0.5394 - auc: 0.5537 - val_loss: 0.6853 - val_accuracy: 0.5447 - val_auc: 0.5669 - lr: 1.0000e-04
Epoch 2/50
650/650 [==============================] - 9s 15ms/step - loss: 0.6852 - accuracy: 0.5479 - auc: 0.5651 - val_loss: 0.6837 - val_accuracy: 0.5526 - val_auc: 0.5736 - lr: 1.2589e-04
Epoch 3/50
650/650 [==============================] - 9s 14ms/step - loss: 0.6813 - accuracy: 0.5577 - auc: 0.5811 - val_loss: 0.6745 - val_accuracy: 0.5845 - val_auc: 0.6116 - lr: 1.5849e-04
Epoch 4/50
111/650 [====>.........................] - ETA: 6s - loss: 0.6764 - accuracy: 0.5667 - auc: 0.6010

KeyboardInterrupt: 

In [ ]:
i

In [5]:
#untuned learning rate for reference 

lstm_dim = 64
dense_dim=64
lstm_dropout=0.15

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=lstm_dim, input_shape=(60,13), dropout=lstm_dropout),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 10))
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


history=model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=50, callbacks=[])


Epoch 1/50
650/650 [==============================] - 8s 10ms/step - loss: 0.6855 - accuracy: 0.5526 - auc: 0.5679 - val_loss: 0.6780 - val_accuracy: 0.5822 - val_auc: 0.6030
Epoch 2/50
650/650 [==============================] - 6s 9ms/step - loss: 0.6730 - accuracy: 0.5812 - auc: 0.6116 - val_loss: 0.6682 - val_accuracy: 0.5947 - val_auc: 0.6386
Epoch 3/50
650/650 [==============================] - 6s 9ms/step - loss: 0.6578 - accuracy: 0.6129 - auc: 0.6457 - val_loss: 0.6721 - val_accuracy: 0.5976 - val_auc: 0.6898
Epoch 4/50
650/650 [==============================] - 6s 9ms/step - loss: 0.6372 - accuracy: 0.6390 - auc: 0.6795 - val_loss: 0.6476 - val_accuracy: 0.6513 - val_auc: 0.7098
Epoch 5/50
650/650 [==============================] - 6s 9ms/step - loss: 0.6132 - accuracy: 0.6668 - auc: 0.7198 - val_loss: 0.5779 - val_accuracy: 0.6954 - val_auc: 0.7670
Epoch 6/50
650/650 [==============================] - 6s 9ms/step - loss: 0.5635 - accuracy: 0.7093 - auc: 0.7800 - val_loss: 0.5

In [ ]:
#tuned learning rate for reference -> diverges

lstm_dim = 64
dense_dim=64
lstm_dropout=0.15

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=lstm_dim, input_shape=(60,13), dropout=lstm_dropout),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 10))
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


history=model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=50, callbacks=[])


In [18]:
#try out callbacks

lstm_dim = 64
dense_dim=64
lstm_dropout=0.15

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=lstm_dim, input_shape=(60,13), dropout=lstm_dropout),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_auc')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auc', min_lr=0.000001,
                        patience=3, mode='max',
                        verbose=1)


checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True, verbose=2)

callbacks = [
    early_stopping,
    reduce_lr,
    model_checkpoint_callback
]

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 10))
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


history=model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=80, callbacks=callbacks)
# The model weights (that are considered the best) are loaded into the model.
model.load_weights(checkpoint_filepath)

pred = model.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred, sequence_ids_test, 'lstm-callbacks')

Epoch 1/80
646/650 [============================>.] - ETA: 0s - loss: 0.6850 - accuracy: 0.5546 - auc: 0.5715
Epoch 1: val_auc improved from -inf to 0.60198, saving model to .\checkpoint
650/650 [==============================] - 8s 11ms/step - loss: 0.6850 - accuracy: 0.5545 - auc: 0.5714 - val_loss: 0.6810 - val_accuracy: 0.5585 - val_auc: 0.6020 - lr: 0.0010
Epoch 2/80
649/650 [============================>.] - ETA: 0s - loss: 0.6778 - accuracy: 0.5685 - auc: 0.5967
Epoch 2: val_auc improved from 0.60198 to 0.63037, saving model to .\checkpoint
650/650 [==============================] - 6s 10ms/step - loss: 0.6778 - accuracy: 0.5685 - auc: 0.5968 - val_loss: 0.6792 - val_accuracy: 0.6034 - val_auc: 0.6304 - lr: 0.0010
Epoch 3/80
649/650 [============================>.] - ETA: 0s - loss: 0.6651 - accuracy: 0.6024 - auc: 0.6334
Epoch 3: val_auc improved from 0.63037 to 0.66095, saving model to .\checkpoint
650/650 [==============================] - 6s 10ms/step - loss: 0.6651 - accura

In [19]:
#try out callbacks

lstm_dim = 64
dense_dim=64
lstm_dropout=0.15

model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=lstm_dim, input_shape=(60,13), dropout=lstm_dropout)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_auc')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auc', min_lr=0.000001,
                        patience=3, mode='max',
                        verbose=1)


checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True, verbose=2)

callbacks = [
    early_stopping,
    reduce_lr,
    model_checkpoint_callback
]

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 10))
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


history=model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=80, callbacks=callbacks)
# The model weights (that are considered the best) are loaded into the model.
model.load_weights(checkpoint_filepath)

pred = model.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred, sequence_ids_test, 'lstm-bidirectional-callbacks')

Epoch 1/80
646/650 [============================>.] - ETA: 0s - loss: 0.6841 - accuracy: 0.5520 - auc: 0.5727
Epoch 1: val_auc improved from -inf to 0.60247, saving model to .\checkpoint
650/650 [==============================] - 12s 14ms/step - loss: 0.6841 - accuracy: 0.5522 - auc: 0.5732 - val_loss: 0.6808 - val_accuracy: 0.5664 - val_auc: 0.6025 - lr: 0.0010
Epoch 2/80
647/650 [============================>.] - ETA: 0s - loss: 0.6729 - accuracy: 0.5834 - auc: 0.6142
Epoch 2: val_auc improved from 0.60247 to 0.65977, saving model to .\checkpoint
650/650 [==============================] - 9s 13ms/step - loss: 0.6729 - accuracy: 0.5833 - auc: 0.6142 - val_loss: 0.6588 - val_accuracy: 0.6178 - val_auc: 0.6598 - lr: 0.0010
Epoch 3/80
647/650 [============================>.] - ETA: 0s - loss: 0.6252 - accuracy: 0.6573 - auc: 0.7071
Epoch 3: val_auc improved from 0.65977 to 0.73545, saving model to .\checkpoint
650/650 [==============================] - 9s 14ms/step - loss: 0.6253 - accur

In [7]:
kernel_size=5
filters=100
dense_dim=32

model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(60,13)),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_auc')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auc', min_lr=0.000001,
                        patience=3, mode='max',
                        verbose=1)


checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True, verbose=2)

callbacks = [
    early_stopping,
    reduce_lr,
    model_checkpoint_callback
]

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 10))
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


history=model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=80, callbacks=callbacks)
# The model weights (that are considered the best) are loaded into the model.
model.load_weights(checkpoint_filepath)

pred = model.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred, sequence_ids_test, 'conv1d-callbacks')


Epoch 1/80
650/650 [==============================] - ETA: 0s - loss: 0.6618 - accuracy: 0.6015 - auc: 0.6393
Epoch 1: val_auc improved from -inf to 0.70600, saving model to .\checkpoint
650/650 [==============================] - 10s 11ms/step - loss: 0.6618 - accuracy: 0.6015 - auc: 0.6393 - val_loss: 0.6323 - val_accuracy: 0.6571 - val_auc: 0.7060 - lr: 0.0010
Epoch 2/80
647/650 [============================>.] - ETA: 0s - loss: 0.6146 - accuracy: 0.6677 - auc: 0.7167
Epoch 2: val_auc improved from 0.70600 to 0.74128, saving model to .\checkpoint
650/650 [==============================] - 7s 10ms/step - loss: 0.6144 - accuracy: 0.6680 - auc: 0.7169 - val_loss: 0.5949 - val_accuracy: 0.6864 - val_auc: 0.7413 - lr: 0.0010
Epoch 3/80
649/650 [============================>.] - ETA: 0s - loss: 0.5766 - accuracy: 0.7018 - auc: 0.7607
Epoch 3: val_auc improved from 0.74128 to 0.78270, saving model to .\checkpoint
650/650 [==============================] - 7s 10ms/step - loss: 0.5766 - accur

In [9]:
kernel_size=5
filters=64
dense_dim=64

dropout_conv=0.4
dropout_dense=0.5

model = tf.keras.Sequential([
   tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(60,13)),
    tf.keras.layers.Dropout(dropout_conv),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.Dropout(dropout_conv),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.Dropout(dropout_conv),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(dropout_dense),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_auc')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auc', min_lr=0.000001,
                        patience=3, mode='max',
                        verbose=1)


checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True, verbose=2)

callbacks = [
    early_stopping,
    reduce_lr,
    model_checkpoint_callback
]

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 10))
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


history=model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=80, callbacks=callbacks)
# The model weights (that are considered the best) are loaded into the model.
model.load_weights(checkpoint_filepath)

pred = model.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred, sequence_ids_test, 'conv1d-callbacks')


Epoch 1/80
650/650 [==============================] - ETA: 0s - loss: 0.6727 - accuracy: 0.5782 - auc: 0.6128
Epoch 1: val_auc improved from -inf to 0.75462, saving model to .\checkpoint
650/650 [==============================] - 11s 15ms/step - loss: 0.6727 - accuracy: 0.5782 - auc: 0.6128 - val_loss: 0.6374 - val_accuracy: 0.6871 - val_auc: 0.7546 - lr: 0.0010
Epoch 2/80
650/650 [==============================] - ETA: 0s - loss: 0.5759 - accuracy: 0.7051 - auc: 0.7741
Epoch 2: val_auc improved from 0.75462 to 0.84397, saving model to .\checkpoint
650/650 [==============================] - 9s 14ms/step - loss: 0.5759 - accuracy: 0.7051 - auc: 0.7741 - val_loss: 0.5449 - val_accuracy: 0.7601 - val_auc: 0.8440 - lr: 0.0010
Epoch 3/80
646/650 [============================>.] - ETA: 0s - loss: 0.5189 - accuracy: 0.7487 - auc: 0.8245
Epoch 3: val_auc improved from 0.84397 to 0.87431, saving model to .\checkpoint
650/650 [==============================] - 10s 15ms/step - loss: 0.5188 - accu

In [10]:
kernel_size=5
filters=64
dense_dim=64
lstm_dim=64

dropout_conv=0.4
dropout_dense=0.5
dropout_lstm=0.5

model = tf.keras.Sequential([
   tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(60,13)),
    tf.keras.layers.Dropout(dropout_conv),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.Dropout(dropout_conv),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.Dropout(dropout_conv),
    tf.keras.layers.LSTM(units=lstm_dim),
    tf.keras.layers.Dropout(dropout_lstm),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(dropout_dense),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_auc')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auc', min_lr=0.000001,
                        patience=3, mode='max',
                        verbose=1)


checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True, verbose=2)

callbacks = [
    early_stopping,
    reduce_lr,
    model_checkpoint_callback
]

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 10))
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


history=model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=80, callbacks=callbacks)
# The model weights (that are considered the best) are loaded into the model.
model.load_weights(checkpoint_filepath)

pred = model.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred, sequence_ids_test, 'conv1d(3)-lstm-callbacks')


Epoch 1/80
650/650 [==============================] - ETA: 0s - loss: 0.6750 - accuracy: 0.5692 - auc: 0.5977
Epoch 1: val_auc improved from -inf to 0.67232, saving model to .\checkpoint
650/650 [==============================] - 17s 21ms/step - loss: 0.6750 - accuracy: 0.5692 - auc: 0.5977 - val_loss: 0.6468 - val_accuracy: 0.6363 - val_auc: 0.6723 - lr: 0.0010
Epoch 2/80
648/650 [============================>.] - ETA: 0s - loss: 0.5973 - accuracy: 0.6819 - auc: 0.7402
Epoch 2: val_auc improved from 0.67232 to 0.83766, saving model to .\checkpoint
650/650 [==============================] - 13s 19ms/step - loss: 0.5969 - accuracy: 0.6822 - auc: 0.7406 - val_loss: 0.5233 - val_accuracy: 0.7337 - val_auc: 0.8377 - lr: 0.0010
Epoch 3/80
649/650 [============================>.] - ETA: 0s - loss: 0.5074 - accuracy: 0.7584 - auc: 0.8320
Epoch 3: val_auc improved from 0.83766 to 0.87316, saving model to .\checkpoint
650/650 [==============================] - 11s 17ms/step - loss: 0.5073 - acc

In [11]:
kernel_size=5
filters=64
dense_dim=64
lstm_dim=64

dropout_conv=0.4
dropout_dense=0.5
dropout_lstm=0.5

model = tf.keras.Sequential([
   tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(60,13)),
    tf.keras.layers.Dropout(dropout_conv),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.Dropout(dropout_conv),
    tf.keras.layers.LSTM(units=lstm_dim),
    tf.keras.layers.Dropout(dropout_lstm),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(dropout_dense),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_auc')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auc', min_lr=0.000001,
                        patience=3, mode='max',
                        verbose=1)


checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True, verbose=2)

callbacks = [
    early_stopping,
    reduce_lr,
    model_checkpoint_callback
]

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 10))
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


history=model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=80, callbacks=callbacks)
# The model weights (that are considered the best) are loaded into the model.
model.load_weights(checkpoint_filepath)

pred = model.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred, sequence_ids_test, 'conv1d(2)-lstm-callbacks')


Epoch 1/80
649/650 [============================>.] - ETA: 0s - loss: 0.6745 - accuracy: 0.5733 - auc: 0.6028
Epoch 1: val_auc improved from -inf to 0.67526, saving model to .\checkpoint
650/650 [==============================] - 14s 19ms/step - loss: 0.6745 - accuracy: 0.5733 - auc: 0.6028 - val_loss: 0.6484 - val_accuracy: 0.6367 - val_auc: 0.6753 - lr: 0.0010
Epoch 2/80
650/650 [==============================] - ETA: 0s - loss: 0.5956 - accuracy: 0.6838 - auc: 0.7423
Epoch 2: val_auc improved from 0.67526 to 0.83291, saving model to .\checkpoint
650/650 [==============================] - 11s 16ms/step - loss: 0.5956 - accuracy: 0.6838 - auc: 0.7423 - val_loss: 0.5115 - val_accuracy: 0.7482 - val_auc: 0.8329 - lr: 0.0010
Epoch 3/80
647/650 [============================>.] - ETA: 0s - loss: 0.5165 - accuracy: 0.7480 - auc: 0.8234
Epoch 3: val_auc improved from 0.83291 to 0.86288, saving model to .\checkpoint
650/650 [==============================] - 11s 17ms/step - loss: 0.5165 - acc

In [12]:
kernel_size=5
filters=64
dense_dim=64
lstm_dim=64

dropout_conv=0.4
dropout_dense=0.5
dropout_lstm=0.5

model = tf.keras.Sequential([
   tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(60,13)),
    tf.keras.layers.Dropout(dropout_conv),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.Dropout(dropout_conv),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=lstm_dim)),
    tf.keras.layers.Dropout(dropout_lstm),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(dropout_dense),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_auc')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auc', min_lr=0.000001,
                        patience=3, mode='max',
                        verbose=1)


checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True, verbose=2)

callbacks = [
    early_stopping,
    reduce_lr,
    model_checkpoint_callback
]

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 10))
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


history=model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=80, callbacks=callbacks)
# The model weights (that are considered the best) are loaded into the model.
model.load_weights(checkpoint_filepath)

pred = model.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred, sequence_ids_test, 'conv1d(2)-lstm(bi)-callbacks')


Epoch 1/80
649/650 [============================>.] - ETA: 0s - loss: 0.6705 - accuracy: 0.5792 - auc: 0.6143
Epoch 1: val_auc improved from -inf to 0.71127, saving model to .\checkpoint
650/650 [==============================] - 19s 24ms/step - loss: 0.6704 - accuracy: 0.5792 - auc: 0.6144 - val_loss: 0.6262 - val_accuracy: 0.6594 - val_auc: 0.7113 - lr: 0.0010
Epoch 2/80
650/650 [==============================] - ETA: 0s - loss: 0.5763 - accuracy: 0.7030 - auc: 0.7682
Epoch 2: val_auc improved from 0.71127 to 0.84610, saving model to .\checkpoint
650/650 [==============================] - 14s 22ms/step - loss: 0.5763 - accuracy: 0.7030 - auc: 0.7682 - val_loss: 0.4877 - val_accuracy: 0.7590 - val_auc: 0.8461 - lr: 0.0010
Epoch 3/80
648/650 [============================>.] - ETA: 0s - loss: 0.5026 - accuracy: 0.7591 - auc: 0.8341
Epoch 3: val_auc improved from 0.84610 to 0.87004, saving model to .\checkpoint
650/650 [==============================] - 14s 22ms/step - loss: 0.5025 - acc

In [13]:
kernel_size=5
filters=64
dense_dim=64
lstm_dim=64

dropout_conv=0.4
dropout_dense=0.5
dropout_lstm=0.5

model = tf.keras.Sequential([
   tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(60,13)),
    tf.keras.layers.Dropout(dropout_conv),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.LSTM(units=lstm_dim),
    tf.keras.layers.Dropout(dropout_lstm),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(dropout_dense),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_auc')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auc', min_lr=0.000001,
                        patience=3, mode='max',
                        verbose=1)


checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True, verbose=2)

callbacks = [
    early_stopping,
    reduce_lr,
    model_checkpoint_callback
]

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 10))
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


history=model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=80, callbacks=callbacks)
# The model weights (that are considered the best) are loaded into the model.
model.load_weights(checkpoint_filepath)

pred = model.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred, sequence_ids_test, 'conv1d(1)-lstm-callbacks')


Epoch 1/80
648/650 [============================>.] - ETA: 0s - loss: 0.6720 - accuracy: 0.5809 - auc: 0.6074
Epoch 1: val_auc improved from -inf to 0.66569, saving model to .\checkpoint
650/650 [==============================] - 14s 19ms/step - loss: 0.6720 - accuracy: 0.5810 - auc: 0.6074 - val_loss: 0.6621 - val_accuracy: 0.6013 - val_auc: 0.6657 - lr: 0.0010
Epoch 2/80
649/650 [============================>.] - ETA: 0s - loss: 0.6364 - accuracy: 0.6392 - auc: 0.6707
Epoch 2: val_auc improved from 0.66569 to 0.69998, saving model to .\checkpoint
650/650 [==============================] - 11s 17ms/step - loss: 0.6363 - accuracy: 0.6392 - auc: 0.6708 - val_loss: 0.6130 - val_accuracy: 0.6615 - val_auc: 0.7000 - lr: 0.0010
Epoch 3/80
650/650 [==============================] - ETA: 0s - loss: 0.6175 - accuracy: 0.6608 - auc: 0.7006
Epoch 3: val_auc improved from 0.69998 to 0.72405, saving model to .\checkpoint
650/650 [==============================] - 11s 17ms/step - loss: 0.6175 - acc

In [14]:
kernel_size=5
filters=64
dense_dim=64
lstm_dim=64

dropout_conv=0.4
dropout_dense=0.5
dropout_lstm=0.5

model = tf.keras.Sequential([
   tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(60,13)),
    tf.keras.layers.Dropout(dropout_conv),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.GRU(units=lstm_dim),
    tf.keras.layers.Dropout(dropout_lstm),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(dropout_dense),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_auc')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auc', min_lr=0.000001,
                        patience=3, mode='max',
                        verbose=1)


checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True, verbose=2)

callbacks = [
    early_stopping,
    reduce_lr,
    model_checkpoint_callback
]

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 10))
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


history=model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=80, callbacks=callbacks)
# The model weights (that are considered the best) are loaded into the model.
model.load_weights(checkpoint_filepath)

pred = model.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred, sequence_ids_test, 'conv1d(1)-gru-callbacks')


Epoch 1/80
650/650 [==============================] - ETA: 0s - loss: 0.6792 - accuracy: 0.5666 - auc: 0.5904
Epoch 1: val_auc improved from -inf to 0.65497, saving model to .\checkpoint
650/650 [==============================] - 14s 18ms/step - loss: 0.6792 - accuracy: 0.5666 - auc: 0.5904 - val_loss: 0.6627 - val_accuracy: 0.5959 - val_auc: 0.6550 - lr: 0.0010
Epoch 2/80
650/650 [==============================] - ETA: 0s - loss: 0.6434 - accuracy: 0.6289 - auc: 0.6618
Epoch 2: val_auc improved from 0.65497 to 0.68994, saving model to .\checkpoint
650/650 [==============================] - 11s 17ms/step - loss: 0.6434 - accuracy: 0.6289 - auc: 0.6618 - val_loss: 0.6248 - val_accuracy: 0.6556 - val_auc: 0.6899 - lr: 0.0010
Epoch 3/80
648/650 [============================>.] - ETA: 0s - loss: 0.6217 - accuracy: 0.6566 - auc: 0.6923
Epoch 3: val_auc improved from 0.68994 to 0.71831, saving model to .\checkpoint
650/650 [==============================] - 10s 15ms/step - loss: 0.6217 - acc

In [ ]:
kernel_size=5
filters=64
dense_dim=64
lstm_dim=64
batch_size=128

dropout_conv=0.4
dropout_dense=0.5
dropout_lstm=0.5

model = tf.keras.Sequential([
   tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(60,13)),
    tf.keras.layers.Dropout(dropout_conv),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.GRU(units=lstm_dim),
    tf.keras.layers.Dropout(dropout_lstm),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dropout(dropout_dense),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_auc')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auc', min_lr=0.000001,
                        patience=3, mode='max',
                        verbose=1)


checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True, verbose=2)

callbacks = [
    early_stopping,
    reduce_lr,
    model_checkpoint_callback
]

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 10))
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


history=model.fit(util.preprocess_sensor_data(sensor_data_trainval), np.array(labels_trainval), validation_split=0.2, epochs=80, callbacks=callbacks, batch_size=batch_size)
# The model weights (that are considered the best) are loaded into the model.
model.load_weights(checkpoint_filepath)

pred = model.predict(util.preprocess_sensor_data(sensor_data_test))
util.generate_submission(pred, sequence_ids_test, 'conv1d(1)-gru-callbacks')
